In [8]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as sts
import seaborn as sns
import datetime
%matplotlib inline


nyc_longitude_west = -74.25559
nyc_longitude_east = -73.70001
nyc_latitude_north = 40.91553
nyc_latitude_south = 40.49612

data = pd.read_csv("yellow_tripdata_2016-05.csv", 
                   parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'], 
                   usecols=['tpep_pickup_datetime', 'tpep_dropoff_datetime',
                            'passenger_count', 'trip_distance', 
                            'pickup_longitude', 'pickup_latitude'])
data.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude
0,2016-05-01,2016-05-01 00:17:31,1,3.60,-73.985901,40.768040
1,2016-05-01,2016-05-01 00:07:31,1,1.68,-73.991577,40.744751
2,2016-05-01,2016-05-01 00:07:01,6,1.09,-73.993073,40.741573
3,2016-05-01,2016-05-01 00:19:47,1,4.21,-73.991943,40.684601
4,2016-05-01,2016-05-01 00:06:39,1,0.56,-74.005280,40.740192


In [9]:
data = data[((data.tpep_dropoff_datetime - data.tpep_pickup_datetime) > datetime.timedelta(0)) &
            (data.passenger_count > 0) & (data.trip_distance > 0) &
            (data.pickup_latitude < nyc_latitude_north) & (data.pickup_latitude > nyc_latitude_south) &
            (data.pickup_longitude < nyc_longitude_east) & (data.pickup_longitude > nyc_longitude_west)]
data.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude
0,2016-05-01,2016-05-01 00:17:31,1,3.60,-73.985901,40.768040
1,2016-05-01,2016-05-01 00:07:31,1,1.68,-73.991577,40.744751
2,2016-05-01,2016-05-01 00:07:01,6,1.09,-73.993073,40.741573
3,2016-05-01,2016-05-01 00:19:47,1,4.21,-73.991943,40.684601
4,2016-05-01,2016-05-01 00:06:39,1,0.56,-74.005280,40.740192


In [10]:
regions = pd.read_csv("regions.csv", delimiter=';')
regions.head()

,region,west,east,south,north
0,1,-74.25559,-74.244478,40.496120,40.504508
1,2,-74.25559,-74.244478,40.504508,40.512896
2,3,-74.25559,-74.244478,40.512896,40.521285
3,4,-74.25559,-74.244478,40.521285,40.529673
4,5,-74.25559,-74.244478,40.529673,40.538061


In [11]:
data['timestamp'] = data.tpep_pickup_datetime.apply(lambda x: x.value)
data.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,timestamp
0,2016-05-01,2016-05-01 00:17:31,1,3.60,-73.985901,40.768040,1462060800000000000
1,2016-05-01,2016-05-01 00:07:31,1,1.68,-73.991577,40.744751,1462060800000000000
2,2016-05-01,2016-05-01 00:07:01,6,1.09,-73.993073,40.741573,1462060800000000000
3,2016-05-01,2016-05-01 00:19:47,1,4.21,-73.991943,40.684601,1462060800000000000
4,2016-05-01,2016-05-01 00:06:39,1,0.56,-74.005280,40.740192,1462060800000000000


In [12]:
data['region'] = (data.pickup_longitude - 1)*50.0 + data.pickup_latitude



AttributeError: 'DataFrame' object has no attribute 'region'

In [ ]:
agg_data = sts.binned_statistic_2d(data.region, data.timestamp, None, 'count', 
                                   bins=[np.arange(1, 2502), times])